In [33]:
from scipy.sparse import *
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import datasets
from importlib import reload as rl
import aux
import json  # salvar benchmarks
import timeit
import scipy


In [3]:
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]

news_bunch = datasets.fetch_20newsgroups(subset='all',
                              remove=('headers', 'footers', 'quotes'))
 
X_news_all = TfidfVectorizer().fit_transform(news_bunch.data)

Y_news_all = news_bunch.target


#### Inicializa corpus e também a versão Tfidf dele

In [40]:
def shrink_cp():
    global cp
    cp = [ 
        'o sapo nao lava op peh nao lava porque nao quer', 
        'ele mora la na lagoa não lava o pé porque não quer', 
        'mas que chulé!', 
        'Ele mora lá na lagoa não lava o pé porque não quer' 
    ]
    _complementary_cp()
def expand_cp():
    global cp
    cp = news_bunch.data
    _complementary_cp()

def _complementary_cp():
    global mcp, mcopy, mc0, mc0ar
    mcp = TfidfVectorizer().fit_transform(cp)
    mcopy = mcp.copy()
    mc0 = mcp[0]
    mc0ar = mc0.toarray()

In [46]:
def time_function(func, *args, **kargs):
    def wrap():
        if kargs:
            return func(*args, kargs)
        else:
            return func(*args)
        
    return timeit.timeit(wrap)

def loop_time_function(n, func, *args, **kargs):
    """" Executa n vezes a função func, passando args e kargs como argumento.
    Seu retorno é a soma dos tempos de execução de cada chamada à função.
    """
    return sum( 
        timeit.repeat(stmt=f'func(*args, **kargs)', 
                      number=1, repeat=n, setup=f'func = {func}', globals=globals())
#         (time_function(func, *args, **kargs) for _ in range(n) ) 
    )
loop_time_function(1, print, 3)

SyntaxError: invalid syntax (<timeit-src>, line 1)

In [6]:
expand_cp()

## Código para subtração manual, elemento a elemento
    + executou de forma consistentemente mais lenta que as versões vetorizadas

In [7]:
def manual_sub(cpy=mcopy, dest=mcp, idx = 0):
    global mc0
    mcp[0] - mc0
    mcp[1] - mc0
    mcp[2] - mc0
    mcp[3] - mc0

## Subtração ao se converter tudo para np.array foi a mais veloz
    
    Contudo, inviável para conjuntos maiores de dados (alta complexidade de espaço) conforme demonstrado ao se tentar utilizar essa estratégia com o dataset de notícias
    20newsgroup.
```python
# PROBLEMA: COMPLEXIDADE DE ESPAÇO
def numpy_array_sub(cpy=mcopy, dest=mcp, idx = 0):
    cpy.toarray() - cpy[idx].toarray()
```

In [8]:
# PROBLEMA: COMPLEXIDADE DE ESPAÇO
def numpy_array_sub(cpy=mcopy, dest=mcp, idx = 0):
    return mcp.toarray() - mcp[idx].toarray()


In [9]:
# Define algumas funções candidatas a substituir a subtração com conversão direta
# para array a partir de matriz esparsa.
mc0ToArray = mc0.toarray()
# Ok.
def numpy_gambs_sub(cpy=mcopy, dest=mcp, idx = 0):
    return np.array([*mcp]) - np.array(mcp.shape[0]*[mcp[idx]])

_sub = lambda a,b: a - b
_vsub = np.frompyfunc(_sub, 2, 1)
# Ok, however in 'ramdom' order (correct submatrix)
def vectorizedpy_sub(cpy=mcopy, dest=mcp, idx = 0):
    return _vsub(np.array([*mcp]), mc0)

# Ok. Same as 'vectorized_sub' 
def vectorized_gambs(cpy=mcopy, dest=mcp, idx = 0):
    return _vsub([*mcp], mc0)

# TESTANTO
def _sub_stupid(a, b):
    return a - b

_stupid_vec = np.frompyfunc(_sub_stupid, 2, 1)
def stupid_method(cpy=mcopy, dest=mcp, idx = 0):
    mc0ar = np.array(mc0)
    return _stupid_vec([*mcp], mc0ar)

# IDEIA ORIGINALL
_to_sub = np.arange( 0, mcp.shape[0], dtype=csr_matrix )

_to_sub[np.array(range(mcp.shape[0]))] = mc0
def original_idea(cpy=mcopy, dest=mcp, idx = 0):
    return [*mcp] - _to_sub

# Parece LENTO
def original_lil(cpy=mcopy, dest=mcp, idx = 0):
    return [*mcp.tolil()] -np.array([mc0])
# Ok. Simples e FUNCIONA
def simple_sub(cpy=mcopy, dest=mcp, idx = 0):
    return np.array([*mcp]) - [mcp[0]]

In [10]:
print(mcp[0].shape)
print(mcp.shape[0])

_sub = lambda a,b: a - b
_vsub = np.frompyfunc(_sub, 2, 1)
# Ok, however in 'ramdom' order (correct submatrix)
def vectorizedpy_sub(cpy=mcopy, dest=mcp, idx = 0):
    return _vsub(np.array([*mcp]), mc0)

# vectorized_sub()
# vectorize_toarray()
# np.array([*mcp])

(1, 134410)
18846


In [11]:
benchmark_small = {}
benchmark_big = {}